<center><img src = "https://images.unsplash.com/photo-1502033491742-0e11fb057e16?q=80&w=1332&auto=format&fit=crop&ixlib=rb-4.1.0&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D" width="1200" height="600"/></center>

<center> photo credit: Julien Andrieux - Unsplash </center>

# Movie Data - Programmatic Data Handling
## *Harry Webber*

## 1. Data Setup

Firstly, we need to import the python libraries that we will need for the data handling and visualisation.

In [ ]:
import numpy as np                # numpy! for mathematical operations
import seaborn as sns             # creating visualisations!
import matplotlib.pyplot as plt   # customising visualisations!
import pandas as pd               # dataframes & data analysis! -- general data use!

We then import the dataset using pandas and save it as our dataframe for the data handling.

In [ ]:
## Create a dataframe called df and read in the csv from the local file
df = pd.read_csv('TMDB_movies.csv')

## 2. Initial Dataframe Inspection

### 2.1 Shape of the dataframe

We inspect the shape of the dataframe (how many rows and columns) and the first rows of the dataframe (using the `.head()` method) to understand what columns there are and what information is present in the data.

In [ ]:
df.shape

> We find that there are 4803 rows/entries and 20 columns in the dataframe. 

In [ ]:
df.head()

My initial impressions, having inspected the dataframe are as follows:
1. The columns below are formatted in a manner that is not yet useful - seems to be some kind of nested dictionary...
    * 'genres'
    * 'keywords'
    * 'production_companies'
    * 'production_countries'
    * 'spoken_languages'
2. We also want to inspect the number of null values in each column of the dataframe.

### 2.2 Check for Missing Data

In [ ]:
df.isnull().sum()    ## count the nulls in each column of the dataframe

In [ ]:
def null_vals(dataframe):                                                 ## This function is copied from earlier in the training programme to show the percentage of missing data by column 
    null_vals = dataframe.isnull().sum()                                  ## How many nulls in each column
    total_cnt = len(dataframe)                                            ## Total entries in the dataframe
    null_vals = pd.DataFrame(null_vals,columns=['null'])                  ## Put the number of nulls in a single dataframe
    null_vals['percent'] = round((null_vals['null']/total_cnt)*100,3)     ## Round how many nulls are there, as %, of the df
    
    return null_vals.sort_values('percent', ascending=False)

null_vals(df)

There are several columns involving null values and we will need to investigate or process them accordingly.
* **'homepage'** has 3091 nulls - this is a large proportion (64%) of the data which we should not simply remove, since it could affect the findings dramatically.
* **'overview'**, **'release_date'**, and **'runtime'** has 3, 1 and 2 nulls, respectively - we should investigate why this is but could remove these entries if required.
* **'tagline'** has 844 null values - ideally we should not remove these (it is nearly 18% of the data) and should find an alternative method for handling these.

I will first deal with the homepage 'column', then the 'tagline' and then the columns which have only 3 or fewer entries missing.

## 3. Null Handling

### 3.1 Standard Fill: 'homepage' data

Since the 'homepage' column only shows a website link to the homepage of each respective film - which I am unlikley to need for the data exploration - I have decided to fill the nulls of that column with a "flag" (a placeholder value) to indicate that the entry is missing data. It appears that the column consists of strings or objects, so once I have double checked the datatype, I can this with a null placeholder value. I decide not to use padding or backfilling here, since there is no consistent relation between the separate entries in the dataframe. The flag I will use is simply 'X' - to show that the 'homepage' data was not available.

In [ ]:
df.dtypes

The datatype is indeed an object (holding website links) which should work using any text string as a placeholder for the missing data, so we go ahead will the flagging process.

In [ ]:
df['homepage'] = df['homepage'].fillna(
    value = 'X', ## our value - this is called 'static filling!' - there is only 1 value
    method = None)

In [ ]:
null_vals(df)  ## we now wish to confirm that there are no remaining NULL values in the 'homepage' column by running the previously defined function

### 3.2 Standard Fill: 'tagline' data

On further inspection of the dataframe, the tagline column simply holds a short tagline for the film - which may have appeared on a poster for example, since there are a large number **(844)** missing entries in this column, it would be very time consuming and inefficient to try to figure them out individually at this stage. Therefore, we will follow the same process that we just carried out for the 'homepage' column, filling the missing values with a placeholder value 'X', for consistent signalling of missing data across the two columns.

In [ ]:
df['tagline'] = df['tagline'].fillna(
    value = 'X', ## our value - this is called 'static filling!' - there is only 1 value
    method = None)

In [ ]:
null_vals(df)  ## we now wish to confirm that there are no remaining NULL values in the 'tagline' column by running the previously defined function

### 3.3 Dropping Nulls: 'overview', 'runtime', 'release_date'

There are multiple options for handling the missing data in these columns.
* I could "custom fill" the runtime and release date with the means of their respective genres, for example.
* I could research and input an overview for those 3 entries.
* I could drop the entries since it only accounts for about 1% of the data.

Since this is just a single day data exploration task - I decide to just drop the remaining nulls in the interest of time.

In [ ]:
df[df['overview'].isnull() | df['runtime'].isnull() | df['release_date'].isnull()]  ## we do a mask to check which entries are missing from these columns

It actually transpires that there is some overlap between the entries which have nulls in these columns, so rather than having to drop up to 6 entries from the dataframe, there are only 4 - **which is great!** We lose less data than expected!

*However, I am concerned that there are entries in other columns which are also missing data - yet are not showing as null since they currently just have punctuation holding the value in their respective cells. This is something that we may need to investigate further later on in the data cleaning process.*

In [ ]:
df.dropna(
    axis = 0, # default is always to drop a row
    how = 'any', # default - if you set to 'all' - would only drop if every value is NULL
    subset = ['overview', 'runtime', 'release_date'],   # subset tells dropna which columns to consider
    inplace = True # makes change permanent
)

We have now dropped these null values and this affected the indexing on the dataframe since some have been removed, so we readjust this by reseting the index values.

In [ ]:
df.reset_index()    # after dropping values to not have 'gaps'

Furthermore, we can now check for any remaining nulls in the initial dataframe... using the same `isnull()` method as before.

In [ ]:
df.isnull().sum()

**The dataframe now appears to be 'clean' - at least there are no null values.**

## 4. Further Data Cleaning

I mentioned the following in the initial dataframe inspection section...
> A number of columns are format in a manner that is not particularly useful - for example, involving extraneous punctuation marks, these include:
**'genres', 'keywords', 'production_companies', 'production_countries', and 'spoken_languages'**

I should now investigate this further before trying to interpret the data, since I have concerns that some of this consists of more missing data.

In [ ]:
df.head()

In [ ]:
## I can't actually see the full text of the cells here since they are too long to be displayed...

pd.set_option('display.max_colwidth', None)  # Shows full content of each cell
df.head()

Upon further inspection, I can see the following...
* The 'genres' column contains information relating to a dictionary of genres, where each genre has its own ID
* The 'keywords' column contains similar information but with a dictionary of keywords
* The 'production_companies' is a dictionary of production companies
* 'production_countries' and 'spoken_languages' are similarly dictionaries

> At this point, I needed some support from my manager in resolving these in a timely manner - he suggested that I could install the library 'ast' - standing for Abstract Syntax Tree - which allows you to format data of given code structures.
> He also provided me with a function ('extract_genres') below, which helps with this process of parsing the data into an appropriate format. 

### 4.1: Formatting the Genres

In [ ]:
import ast
ast.literal_eval(df.genres[0])[0]['name']

In [ ]:
def extract_genres(x):
    ''' Function to extract genres for each datapoint. '''
    x = ast.literal_eval(x) ## Transform '['name', 'id']' back into ['name', 'id']
    Genres = [] ## Empty list to store the genres
    
    # Iterate through each dictionary
    for item in x: ## iterate for each dictionary in our list
        Genres.append(item['name']) ## Grab the 'name' key for each dictionary
        
    # Return the Genres
    return Genres

## Apply the function to genres
df['extracted_genres'] = df['genres'].apply(extract_genres)

In [ ]:
df.head()

### 4.2: Formatting the Keywords

Since this was quite effective - we will try the same thing for the keywords and other columns involving these nested dictionary datatypes.

In [ ]:
def extract_keyword(x):
    ''' Function to extract keywords for each datapoint. '''
    x = ast.literal_eval(x) ## Transform '['name', 'id']' back into ['name', 'id']
    Keywords = [] ## Empty list to store the keywords
    
    # Iterate through each dictionary
    for item in x: ## iterate for each dictionary in our list
        Keywords.append(item['name']) ## Grab the 'name' key for each dictionary
        
    # Return the Keywords
    return Keywords

## Apply the function to keywords column
df['extracted_keywords'] = df['keywords'].apply(extract_keyword)

In [ ]:
df.head() ## check the new column looks appropriate

### 4.3: Formatting the Production Companies

In [ ]:
def extract_company(x):
    ''' Function to extract production companies for each datapoint. '''
    x = ast.literal_eval(x) ## Transform '['name', 'id']' back into ['name', 'id']
    Companies = [] ## Empty list to store the Companies
    
    # Iterate through each dictionary
    for item in x: ## iterate for each dictionary in our list
        Companies.append(item['name']) ## Grab the 'name' key for each dictionary
        
    # Return the Companies
    return Companies

## Apply the function to production_companies column
df['extracted_production_companies'] = df['production_companies'].apply(extract_company)

In [ ]:
df.head()

### 4.4: Formatting the Production Countries

In [ ]:
def extract_country(x):
    ''' Function to extract production countries for each datapoint. '''
    x = ast.literal_eval(x) ## Transform '['name', 'id']' back into ['name', 'id']
    Countries = [] ## Empty list to store the Countries
    
    # Iterate through each dictionary
    for item in x: ## iterate for each dictionary in our list
        Countries.append(item['name']) ## Grab the 'name' key for each dictionary
        
    # Return the Countries
    return Countries

## Apply the function to production_countries column
df['extracted_production_countries'] = df['production_countries'].apply(extract_country)

In [ ]:
df.head()

### 4.5: Formatting the Spoken Languages

In [ ]:
def extract_language(x):
    ''' Function to extract spoken languages for each datapoint. '''
    x = ast.literal_eval(x) ## Transform '['name', 'id']' back into ['name', 'id']
    Languages = [] ## Empty list to store the Languages
    
    # Iterate through each dictionary
    for item in x: ## iterate for each dictionary in our list
        Languages.append(item['name']) ## Grab the 'name' key for each dictionary
        
    # Return the Languages
    return Languages

## Apply the function to spoken_languages column
df['extracted_spoken_languages'] = df['spoken_languages'].apply(extract_language)

In [ ]:
df.head()

### 4.6: Dropping Unwanted Columns

Since the dataframe is getting quite crowded now with 5 additional columns, I will remove the original columns and keep the extracted information for use in data interpretation and visualisation stages.

In [ ]:
df = df.drop(columns=['genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages'])
df.head()

In [ ]:
## I also don't want the 'overview' column as there is a long string of text in this and it will not be particularly useful in the data visualisations I have in mind.

df = df.drop(columns=['overview'])
df.head()

## 5. Data Exploration

### 5.1 Initial Ideas

To get a sense of what the data actually says, I will first use the `.describe()` method to see the summary statistics for each of the numeric datatypes.

In [ ]:
df.describe()

My intial thoughts are as follows...
* It is difficult to understand the **budget** and **revenue** columns since the numbers are large and not instantly simple to interpret - these will need further investigation.
* The summary statistics for ID are probably not particularly helpful.
* I don't fully understand the relationship between things like **vote_average** and **popularity** at this stage, but they seem to be similar metrics - just with vote_average being a score out of 10.
* It may be interesting to filter/mask the popularity measures by those which have received at least a certain threshold of votes in **vote_count**.
* It could also be interesting to investigate the runtime and see what impact this has on other metrics of the films (what film is 338 minutes long?!).

### 5.2 Checking Correlations

> Following this, it makes sense to check the correlations between some of these numeric metric within the dataframe...

In [ ]:
# Firstly, I select the columns which I wish to check for correlation between

corr_values = df[['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count']]   # all numeric columns except 'id'

plt.figure(figsize = (8,8))                                                                      # set the figure size
sns.heatmap(corr_values.corr(),                                                                  # I want to make a heatmap effectively showing a correlation matrix between these columns
            annot = True,
            fmt = '.1%',
            cmap=sns.diverging_palette(0, 240, as_cmap=True),                                    # colour palette set to diverge from red to blue with red indicating negative correlation and blue indicating positive
            vmin = -1, vmax = +1,
            mask = np.triu(corr_values.corr()))                                                  # this masks it as a triangle to not show any duplicates or the diagonal where correlation = 1

plt.title('Fig. 1: Correlation Heatmap', fontsize = 16)                                          # naming the plot
plt.show()

# If I instead wanted to save the chart as a .png file, I would use the following code
# plt.savefig('movies_correlation_heatmap.png', dpi = 300, bbox_inches = 'tight')

> This correlation heatmap has no negative correlation at all - which is an interesting insight in itself - but the correlations do range from being basically trivial to quite strong.
> * There are few surprises in the fact that **budget** is correlated with **revenue** since the film may include bigger 'stars' and have had more promotion.
> * **Revenue** is correlated quite strongly with **popularity**, but there is weak correlation with **vote_average**.
> * In fact, **vote_average** and **popularity** only share a weak correlation themselves which seems strange to me.
>     * Both seem to be related to how good a film is.
>     * However, I suppose a film can be popular without being highly rated.
>     * Many 'blockbusters' or 'big comedy' films are purely for entertainment - but are quite poor aesthetically (this is sometimes even deliberate!)

### 5.3 Looking at Revenue

Since revenue is quite strongly correlated with budget, but also with things like vote_count and even popularity, I investigate the films by revenue first.

> For this section:
> * We define a new metric: 'performance ratio' as 'revenue over budget' (i.e. how many times its budget did it make?)
> * We consider only the most voted films (those with over 5000 votes - top 84 in the dataframe) since they have at least been viewed by a statistically significant number of people.

In [ ]:
df['performance_ratio'] = df['revenue'] / df['budget']     # How many times the film budget did it make in revenue

In [ ]:
most_voted = df[df['vote_count'] > 5000]                   # I am mainly interested here in films which many people have seen / have received a good number of votes

In [ ]:
plt.figure(figsize = (8,8))
sns.regplot(data=most_voted, x='budget', y='revenue', color = "green", scatter_kws={"color": "skyblue"}, line_kws={"color": "red"}, ci = None)

plt.xlabel('Movie Budget (100 Mil.)')
plt.ylabel('Movie Revenue (Bil.)')

plt.xlim(0, )                                                       # Set x-axis limits
plt.ylim(0, )                                                       # Set y-axis limits

plt.title('Fig. 2: Budget-Revenue Scatter Plot', fontsize = 16)     # naming the plot
plt.show()

In [ ]:
plt.figure(figsize = (8,8))
sns.scatterplot(data=most_voted, x='budget', y='performance_ratio', hue = 'revenue')

plt.xlabel('Movie Budget')
plt.ylabel('Performance Ratio (Return on Budget)')                      # How many times the film budget did it make in revenue

plt.xlim(0, )                                                           # Set x-axis limits
plt.ylim(0, )                                                           # Set y-axis limits

plt.title('Fig. 3: Performance against Budget', fontsize = 16)          # naming the plot
plt.show()

In [ ]:
most_voted[most_voted['performance_ratio'] > 70]              # what is the outlier at the very top of this graph??

#### Keypoint 1: Risk-Reward

> * The film with the overall highest revenue (see fig. 2) is **Avatar**.
> * This is also among the highest budget films in the dataframe.
> * Rolling out a film with a large budget is a **big risk** for production companies as there are also many high budget films which don't yield such high revenue - i.e. "commercial flops".
> * When we look at the fig. 3 - we see that there are a large number of lower budget films yielding huge performance ratios (i.e. high percentage profit).
> * We should be careful here, since we have already masked to show only the top 84 most reviewed - many low budget films are unlikely to make it into this bracket.
> * The uppermost point in fig. 3 - making over 70 times the budget of the film - and the film to make if you want to be successful in the film industry is...

##### surprise surprise...

**<center>Star Wars</center>**

<center><img src = "https://images.unsplash.com/photo-1546561892-65bf811416b9?q=80&w=1170&auto=format&fit=crop&ixlib=rb-4.1.0&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D" width="500" height="400"/></center>

<center> photo credit: Tommy Van Kessel - Unsplash </center>

So, if you want to make a commercially successful film, just make a film which will start a massive, decade-spanning franchise -- easy right?? ;)

### 5.4 Which Films are Rated Highly?

Looking back at the correlation heatmap - we also see that **vote_average** is not strongly correlated with any other metric in the dataframe. In fact the correlation coefficient is never more than 0.4 which indicates (at most) moderate correlation. So I am also interested to try to establish more about what makes certain films popular among the people who have voted?

In [ ]:
plt.figure(figsize=(8, 8))
sns.histplot(data=df, x="vote_average", binwidth = 0.2, color = "skyblue", kde=True)
plt.xlim(0, 10)

plt.title('Fig. 4: Distribution of Voting Scores', fontsize = 16)

plt.show()

Looking at the distribution of films here - we see confirm that there is something close to a normal distribution with a mean just over 6. But I am interested at the moment in the highest rated films - so I will look at those with an average rating of over 8. Initially this creates a list of 50 films - however, some of these are quite niche and only have high ratings because they have so few voters anyway (maybe there are a few "mega-fans" but this is not really statistically significant). I want a set of films with the following restrictions...
* Average rating (vote_average) is greater than 8.
* Number of votes (vote_count) is greater than 1000.

In [ ]:
elite_tier = df[(df['vote_average'] > 8) & (df['vote_count'] > 1000)]       # create a subset of the dataframe which only contains the films with ratings over 8 from over 1000 reviews

Instinctively, I am intereseted to see if there is anything that these "elite tier" films have in common. I decided to look at the genres of the films.

In [ ]:
elite_tier['extracted_genres'].value_counts()                               # check the genres of these top rated films

Many of the films here have **'Drama'** as either a main genre - or one in a list of subgenres - this suggests that highly rated films are often in the 'Drama' genre. **Let's explore this a bit further...**

I can do this by splitting the dataframe into those films which do have drama in the list of genres, and those which don't...

In [ ]:
drama_mask = df['extracted_genres'].apply(lambda x: 'Drama' in x)                 # used a lambda function to iterate through the dataframe and find all films which have Drama as a genre
drama_films = df[drama_mask].copy()                                               # used this as a mask on the dataframe

non_drama_mask = df['extracted_genres'].apply(lambda x: 'Drama' not in x)         # repeat for those which do *not* have Drama as a genre
non_drama_films = df[non_drama_mask].copy()

In [ ]:
drama_films.shape                # used this to check the number of films with drama as a genre - 2296

In [ ]:
non_drama_films.shape            # used this to check the number of films without drama as a genre - 2503

In [ ]:
drama_films['is_drama'] = 'drama'                                                # created an extra column showing 'drama' if drama is one of the genres and 'not drama' if it is not
non_drama_films['is_drama'] = 'not drama'

# Concatenate these together into one dataframe for the visualisation -- ignore index argument creates a new consistent index ignoring the originals
drama_df = pd.concat([drama_films, non_drama_films], ignore_index=True)

In [ ]:
plt.figure(figsize=(8, 8))
sns.boxplot(x='is_drama', y='vote_average', data=drama_df, color = 'skyblue', whis = 2.5)            # split the data by the new column 'is_drama' to make comparative boxplots

plt.title('Fig. 5: Box Plots - Drama vs. Not Drama', fontsize = 16)

plt.show()

#### Keypoint 2: Bring the Drama!

> * Films which have 'drama' listed as a genre are consistently rated higher by voters than those which do not.
> * Drama films have a median rating of **6.5** while non-drama films have a median rating of **6.0**.
> * Drama films have a mean rating of **6.4** (1 d.p.) while non-drama have mean rating of **5.8** (1 d.p.).
> * Drama films are also more consistently high rated with a lower range and interquartile range than the non-drama films.
> * **We could say that if you want to make a highly rated film - make a drama** - but we should be careful about conflating the trend here with a direct causation.
> * I do have some concerns about any films which are missing information on genre being bundled together - they may be low-budget / niche films - likely to be low rated.

In [ ]:
non_drama_films['vote_average'].describe()   # for checking summary statistics of non-drama films

In [ ]:
drama_films['vote_average'].describe()   # for checking summary statistics of drama films

## 6. Single Observation Analysis

Since I haven't really yet had an opportunity to analyse the meaning of this 'popularity' metric - I will do so to some extent, by choosing the two "most popular" films according to this metric. They are **Minions** and **Interstellar**.

<center><img src = "https://images.unsplash.com/photo-1515041219749-89347f83291a?q=80&w=1074&auto=format&fit=crop&ixlib=rb-4.1.0&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D" width="600" height="500"/></center>

<center> photo credit: Justin Lim - Unsplash </center>

In [ ]:
df[df['popularity'] == df['popularity'].max()]       # find the most 'popular' film by this metric

In [ ]:
df[df['popularity'] > 700]                            # look at the films with over 700 popularity - I just played around with the numbers here a little after finding the max was minions with 875

When comparing these two films, there are a number of different insights to be made, the table below shows some of the key information and metrics of these two films from the dataframe...

| Metric                                 | Interstellar                                                   | Minions                                     | Insights                                |
| --------                               | --------                                                       | --------                                    | --------                                |
| **Production Companies**               | Paramount - Legendary - Warner Bros. - Syncopy - Lynda Obst    | Universal - Illumination                    | Completely unique production            |
| **Release Date**                       | 2014-11-05                                                     | 2015-06-17                                  | Similar year - different quarters       |
| **Genre(s)**                           | Adventure - Drama - Science Fiction                            | Family - Animation - Adventure - Comedy     | Very different (but both 'Adventure')   |
| **Budget**                             | 165,000,000                                                    | 74,000,000                                  | Interstellar more than double           |
| **Revenue**                            | 675,120,017                                                    | 1,156,730,962                               | Both strong performers                  |
| **Performance Ratio (1 d.p.)**         | 4.1                                                            | 15.6                                        | Minions was a huge commercial success   |
| **Rating (vote_average)**              | 8.1                                                            | 6.4                                         | Interstellar top-rated                  |
| **Number of Reviews (vote_count)**     | 10,867                                                         | 4571                                        | Maybe family/children review less       |

> **<u>Interstellar**
> * Clearly a very **highly rated** film with 8.1 average vote (among top 50 in dataset).
> * Contextually we know that it is more **targeted at adults** (BBFC rated as a 12 [https://www.bbfc.co.uk/release/interstellar-q29sbgvjdglvbjpwwc0zode4nzi]).
> * **Good revenue** making over 4 times its budget - but there was a relatively **high budget** initially.
> * **Cast and crew** costs may have been high with stars like Matthew McConaughey, Anne Hathaway, Jessica Chastain and director: Christopher Nolan [https://www.imdb.com/title/tt0816692/].
>
> **<u>Minions**
> * Not such a high rating (only just above average) and fewer votes. However, children are probably the main target audience and are unlikely to be voting.
> * Being a **family film** and comedy means it has a broad audience - also better **potential for merchandise sales**.
> * **Huge commercial success** with massive revenue and seems even better when we note that it made over 15 times the budget for the film.
> * We should note that this was **not the first film in the 'Despicable Me' universe** and so production companies were likely to be confident in its popularity and success.
>
> **<u>Overall Comparison**
> * **Commercially, Minions was the more successful film** in yielding massive revenue from a budget less than half that of Interstellar.
> * **From a ratings perspective: Interstellar appears to have considerably higher reviews** and is in fact considered one of the best films in the dataset.
> * Whilst they are both very different films stylistically, it is easy to see why both were popular in their own way.
> * Also interesting to note that Minions was released in the Summer window (for school holidays?).
> * Whereas, Interstellar was released in November (ahead of Christmas and potentially considering awards season).